In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


import trax_util as trx

import importlib



import sys
# import modules in other directories
sys.path.append('/glade/work/juliob/PyRegridding/Regridder/')


import esmfRegrid as erg


importlib.reload( trx )


## create ESMF regridding objects -- takes a while

In [ ]:
scrip_dir = '/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
dst_scrip = scrip_dir + 'fv0.23x0.31_141008.nc'
src_scrip = scrip_dir + 'ne120np4_pentagons_100310.nc'

dst1d_scrip = scrip_dir + 'fv0.9x1.25_141008.nc'

#ne120 ==> latlon 0.25 degree
regrd, srcf, dstf = erg.Regrid(srcScrip = src_scrip , 
                                srcType  = 'mesh'  ,
                                dstScrip = dst_scrip ,
                                dstType  = 'grid'   )
    
#ne120 ==> latlon 1 degree
regrd1d, srcf1d, dstf1d = erg.Regrid(srcScrip = src_scrip , 
                                srcType  = 'mesh'  ,
                                dstScrip = dst1d_scrip ,
                                dstType  = 'grid'   )
    


## Take JAS Mean of ne120 3hrly data .... TAKES FOREVER !!!

In [ ]:
drcQd='/glade/campaign/cgd/ccr/nanr/HPSS/tokeep/TIMESLICE/atm/proc/tseries/hourly3/PRECT/'
prcQd=0.  # []
#for y in np.arange( start=1980,stop=2013):
Nyr=0
for y in np.arange( start=1980,stop=2013):
    yA = str(y).zfill(4)
    fQd='f.e13.FAMIPC5.ne120_ne120.1979_2012.001.cam.h4.PRECT.'+yA+'010100Z-'+yA+'123121Z.nc'
    print(fQd)
    dsQd=xr.open_dataset(drcQd+fQd)
    Iyyyymm=dsQd.date.values.astype(int)
    oo=np.where( (Iyyyymm >= y*10_000+ 700 ) & (Iyyyymm < y*10_000+ 1000 ) )  # get Jun-Aug-Sep 
    print(np.shape(oo[0]))
    print( Iyyyymm[oo[0][0]],Iyyyymm[oo[0][-1]])
    #prcQd_0 = dsQd.PRECT.values
    prcQd_0 = dsQd.PRECT[oo[0][:],:].values
    prcQd = prcQd + np.average( prcQd_0 , axis=0 )      
    Nyr=Nyr+1

In [ ]:
lonQd=dsQd.lon.values
latQd=dsQd.lat.values
prcQd = prcQd / Nyr

In [ ]:
prcQd_yx = erg.HorzRG(aSrc = prcQd , 
                regrd = regrd , 
                srcField=srcf , 
                dstField=dstf , 
                srcGridkey='c' ,
                dstGridkey='yx' ) 


In [ ]:
prc1d_yx = erg.HorzRG(aSrc = prcQd , 
                regrd = regrd1d , 
                srcField=srcf1d , 
                dstField=dstf1d , 
                srcGridkey='c' ,
                dstGridkey='yx' ) 


In [ ]:
print(np.shape(prcQd_yx))
Scr=xr.open_dataset( dst_scrip )
latQd_yx = Scr.grid_center_lat.values.reshape( (768, 1152 ) )
lonQd_yx = Scr.grid_center_lon.values.reshape( (768, 1152 ) )
#plt.contour(latQd_yx)
#plt.contour(lonQd_yx)


In [ ]:
print(np.shape(prc1d_yx))
ny1d,nx1d = np.shape(prc1d_yx)
Scr=xr.open_dataset( dst1d_scrip )
lat1d_yx = Scr.grid_center_lat.values.reshape( (ny1d, nx1d ) )
lon1d_yx = Scr.grid_center_lon.values.reshape( (ny1d, nx1d ) )
#plt.contour(latQd_yx)
#plt.contour(lonQd_yx)


In [ ]:
preclv=np.linspace(0,32,num=16+1)
projPC = ccrs.Robinson(central_longitude=0.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

d01=prcQd_yx*86400.*1000.
ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )
ax1.coastlines(resolution='110m',color='black',linewidth=4)
ax1.contourf( lonQd_yx, latQd_yx, d01 ,levels=preclv,cmap='Blues',transform=data_crs )
ax1.spines['geo'].set_linewidth(4)  # change as needed


In [ ]:
preclv=np.linspace(0,24,num=16+1)
projPC = ccrs.Robinson(central_longitude=180.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

d01=prc1d_yx*86400.*1000.
ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )
ax1.coastlines(resolution='110m',color='black',linewidth=4)
po=ax1.contourf( lon1d_yx, lat1d_yx, d01 ,levels=preclv,cmap='Blues',transform=data_crs )
ax1.spines['geo'].set_linewidth(4)  # change as needed

cax = plt.axes([0.93, 0.25, 0.01, 0.5])
cbar=plt.colorbar(po,cax=cax,orientation='vertical')
cbar.ax.tick_params(labelsize=20)
ax1.set_title('CESM1 ne120 Jul-Aug-Sep climo',fontsize=32)

In [ ]:
## Should be done 
## np.savez('arrays.npz', arr1=arr1, arr2=arr2, arr3=arr3)
npSaveFile = '/glade/work/juliob/NumPySaves/ne120-JAS-e01-1980-2012.npz'
np.savez( npSaveFile , prec=prcQd, lat=latQd, lon=lonQd )